In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

from torch.utils.data import Dataset, DataLoader
import os
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

import wandb

# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
# import torchtext.transforms as T
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [ ]:
FOLDER_PATH = '/content/drive/MyDrive/Deep Learning/ex3_305673212_312349509/FashionMNIST'
if (os.path.exists(FOLDER_PATH)):
  path = FOLDER_PATH
else:
  path = "data" #for git runs

In [ ]:
#hyper parameters
#TODO: the WGAN paper states lr= 5e-5, should we use it?
lr = 1e-4 #as defined in improve_wgan_training github repo
epochs = 40
DIM = 64
CRITIC_ITERS = 5 # How many critic iterations per generator iteration
BATCH_SIZE = 64 # Batch size
ITERS = 200000 # How many generator iterations to train for
input_img_w = 28 #FashionMNIST Width
input_img_h = 28 #FashionMNIST Hight
in_features = 1
latent_space = 128
kerenel_size = 5
OUTPUT_DIM = input_img_w*input_img_h*in_features # Number of pixels in FashionMNIST (28*28*1)


# W&B
wandb.init(project="ex3", entity="danielshamir")
wandb.config.lr = lr
wandb.config.epochs = epochs
wandb.config.DIM = DIM
wandb.config.CRITIC_ITERS = CRITIC_ITERS
wandb.config.BATCH_SIZE = BATCH_SIZE
wandb.config.ITERS = ITERS
wandb.config.input_img_w = input_img_w
wandb.config.input_img_h = input_img_h
wandb.config.in_features = in_features
wandb.config.latent_space = latent_space
wandb.config.kerenel_size = kerenel_size
wandb.config.OUTPUT_DIM = OUTPUT_DIM




In [1]:
! pip install wandb
import wandb

  Using cached wandb-0.17.5-py3-none-win_amd64.whl.metadata (10 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached sentry_sdk-2.11.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached setproctitle-1.3.3-cp310-cp310-win_amd64.whl.metadata (10 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
Using cached wandb-0.17.5-py3-none-win_amd64.whl (6.7 MB)
Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
Using cached GitPython-3.1.43-py3-none-any.whl (207 kB)
Using cached sentry_sdk-2.11.0-py2.py3-none-any.whl (303 kB)
Using cached setproctitle-1.3.3-cp310-cp310-win_amd64.whl (11 kB)
Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
Using cached smmap-5.0.1-py3-none-any.whl (24 kB)


# Data Preprocessing
Data loaders

In [ ]:
train_data_raw = open(f'{path}/ptb.train.txt', 'r').read()
test_data_raw = open(f'{path}/ptb.test.txt', 'r').read()
valid_data_raw = open(f'{path}/ptb.valid.txt', 'r').read()
data =  train_data_raw + ' ' + test_data_raw + ' ' + valid_data_raw

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

test_data_iter = iter(test_loader)
train_data_iter = iter(train_loader)
valid_data_iter = iter(valid_loader)

#Network Architecture
We will implemnet gernerator and descriminator/critic as defined in "Improved Training of Wasserstein GANs" papaer for CIFAR10, then we will make adjustments to be compatible to FashuionMNIST data set

MNIST/ Fashion MNIST Input Dimensionsimages are 28x28 grayscale images, while CIFAR-10 images are 32x32 RGB images. We will modify the input and output layers of the networks accordingly.



In [ ]:

#Input to Generator is noise which can be random or not, nosie dimention is 128

def Generator(n_samples, noise=None):
  def __init__(self, dim):
    super(Generator, self).__init__()
    self.dim = dim
    self.linear = nn.Linear(latent_space, 4*4*4*dim)
    self.bn1 = nn.BatchNorm1d(4*4*4*dim)
    self.deconv2 = nn.ConvTranspose2d(4*dim, 2*dim, kernel_size=5, stride=2, padding=2, output_padding=1)
    self.bn2 = nn.BatchNorm2d(2*dim)
    self.deconv3 = nn.ConvTranspose2d(2*dim, dim, kernel_size=5, stride=2, padding=2, output_padding=1)
    self.bn3 = nn.BatchNorm2d(dim)
    self.deconv4 = nn.ConvTranspose2d(dim, 3, kernel_size=5, stride=2, padding=2, output_padding=1)


  def forward(self, n_samples, noise=None):
    if noise is None:
        noise = torch.randn(n_samples, latent_space)

    output = self.linear(noise)
    output = self.bn1(output)
    output = F.relu(output)
    output = output.view(-1, 4*self.dim, 4, 4)

    output = self.deconv2(output)
    output = self.bn2(output)
    output = F.relu(output)

    output = self.deconv3(output)
    output = self.bn3(output)
    output = F.relu(output)

    output = self.deconv4(output)
    output = torch.tanh(output)

    return output.view(-1, OUTPUT_DIM)


class Discriminator(nn.Module):
  def __init__(self, in_features, input_img_w, input_img_h, DIM, kernel_size):
      super(Discriminator, self).__init__()
      self.in_features = in_features
      self.input_img_w = input_img_w
      self.input_img_h = input_img_h
      self.DIM = DIM
      self.kernel_size = kernel_size

      self.conv1 = nn.Conv2d(in_features, DIM, kernel_size, stride=2)
      self.conv2 = nn.Conv2d(DIM, 2*DIM, kernel_size, stride=2)
      self.bn2 = nn.BatchNorm2d(2*DIM)
      self.conv3 = nn.Conv2d(2*DIM, 4*DIM, kernel_size, stride=2)
      self.bn3 = nn.BatchNorm2d(4*DIM)
      self.fc = nn.Linear(4*4*4*DIM, 1)

  def forward(self, inputs):
      output = inputs.view(-1, self.in_features, self.input_img_w, self.input_img_h)

      output = F.leaky_relu(self.conv1(output), negative_slope=0.2)

      output = F.leaky_relu(self.bn2(self.conv2(output)), negative_slope=0.2)

      output = F.leaky_relu(self.bn3(self.conv3(output)), negative_slope=0.2)

      output = output.view(-1, 4*4*4*self.DIM)
      output = self.fc(output)

      return output.view(-1)


#Lost Function/ Inception

In [ ]:

class GAN:
  def __init__(self, dim, mode, train_loader):
    self.generator = Generator(dim)
    self.discriminator = Discriminator(in_features, input_img_w, input_img_h, DIM, kernel_size)
    self.mode = mode
    self.gen_optimizer = optim.RMSprop(self.generator.parameters(), lr=5e-5)
    self.disc_optimizer = optim.RMSprop(self.discriminator.parameters(), lr=5e-5)
    self.setup_optimizers()
    self.train_loader = train_loader

    # NADAV/REUT: Do we need to defrenciate the optimizers?

    #def setup_optimizers(self):
    #     if self.mode == 'wgan':
    #         self.gen_optimizer = optim.RMSprop(self.generator.parameters(), lr=5e-5)
    #         self.disc_optimizer = optim.RMSprop(self.discriminator.parameters(), lr=5e-5)
    #     elif self.mode == 'dcgan':
    #         self.gen_optimizer = optim.Adam(self.generator.parameters(), lr=2e-4, betas=(0.5, 0.999))
    #         self.disc_optimizer = optim.Adam(self.discriminator.parameters(), lr=2e-4, betas=(0.5, 0.999))

    def inf_train_gen(self):
        while True:
            for images, _ in self.train_loader:
                yield images

    def generate_image(self, frame):
        fixed_noise_128 = torch.randn(128, 128).to(device)
        samples = self.generator(128, fixed_noise_128).detach().cpu().numpy()
        samples = ((samples + 1.) * (255. / 2)).astype('int32')
        save_image(torch.tensor(samples).view(128, 3, 32, 32), 'samples_{}.jpg'.format(frame))

    def get_inception_score(self):
        samples_100 = self.generator(100).detach().cpu().numpy()
        all_samples = []
        for _ in range(10):
            all_samples.append(samples_100)
        all_samples = np.concatenate(all_samples, axis=0)
        all_samples = ((all_samples + 1.) * (255. / 2)).astype('int32')
        all_samples = all_samples.reshape((-1, 3, 32, 32)).transpose(0, 2, 3, 1)
        return lib.inception_score.get_inception_score(list(all_samples))



    def train(self):
        for iteration in range(self.iters):
            start_time = time.time()
            # Train generator
            if iteration > 0:
                self.gen_optimizer.zero_grad()
                fake_data = self.generator(self.batch_size)
                disc_fake = self.discriminator(fake_data)
                gen_cost = -torch.mean(disc_fake) if self.mode == 'wgan' else F.binary_cross_entropy_with_logits(disc_fake, torch.ones_like(disc_fake))
                gen_cost.backward()
                self.gen_optimizer.step()

            # Train critic
            disc_iters = 1 if self.mode == 'dcgan' else self.critic_iters
            for _ in range(disc_iters):
                _data = next(self.inf_train_gen()).to(device)
                self.disc_optimizer.zero_grad()
                disc_real = self.discriminator(_data)
                fake_data = self.generator(self.batch_size)
                disc_fake = self.discriminator(fake_data)

                disc_cost = torch.mean(disc_fake) - torch.mean(disc_real) if self.mode == 'wgan' else \
                            (F.binary_cross_entropy_with_logits(disc_fake, torch.zeros_like(disc_fake)) +
                             F.binary_cross_entropy_with_logits(disc_real, torch.ones_like(disc_real))) / 2.
                disc_cost.backward()
                self.disc_optimizer.step()

                if self.mode == 'wgan':
                    for p in self.discriminator.parameters():
                        p.data.clamp_(-0.01, 0.01)

            # Logging
            print(f"Iteration {iteration}, Disc Cost: {disc_cost.item()}, Time: {time.time() - start_time}")

            # Calculate inception score every 1K iters
            if iteration % 1000 == 999:
                inception_score = self.get_inception_score()
                print(f"Inception Score: {inception_score[0]}")

            # Generate samples every 100 iters
            if iteration % 100 == 99:
                self.generate_image(iteration)







